In [1]:
#필요한 모듈 import
import numpy as np
from sklearn.mixture import GaussianMixture
import glob
import os
import wave
from scipy.io import wavfile
import pickle
import matplotlib.pyplot as plt
import librosa
import librosa.effects as effects
import librosa.util
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
import joblib

#학습용 코드에서 패딩할 때 사용한 max_size와 전처리에 사용한 lda 파라미터를 가져옴
%store -r max_size
%store -r lda

# # CTL 파일 경로
# ctl_file = '202301ml_fmcc/fmcc_test900.ctl'

# # RAW 파일이 저장된 폴더 경로
# raw_folder = '202301ml_fmcc/raw16k/test/'

# # WAV 파일이 저장될 폴더 경로
# wav_folder = '202301ml_fmcc/raw16k/test/wavtest/'

#.ctl파일이 있는 경로와 .raw파일이 있는 폴더 명과 변환한 wav파일을 저장할 폴더 경로를 지정
ctl_file = input("읽을 .ctl파일의 경로를 입력하세요 : ")
raw_folder = input(".raw파일이 저장된 폴더 경로를 입력하세요 : ")
wav_folder = input("위의 .raw파일이 입력된 폴더 경로에 wavtest/를 추가하여 입력하세요 : ")

읽을 .ctl파일의 경로를 입력하세요 : 202301ml_fmcc/fmcc_test900.ctl
.raw파일이 저장된 폴더 경로를 입력하세요 : 202301ml_fmcc/raw16k/test/
위의 .raw파일이 입력된 폴더 경로에 wavtest/를 추가하여 입력하세요 : 202301ml_fmcc/raw16k/test/wavtest/


In [2]:
# RAW 파일을 WAV 파일로 변환하는 함수
def convert_raw_to_wav(raw_filepath, wav_filepath):
    sample_width = 2  # 16비트 샘플
    sample_rate = 16000  # 샘플링 속도
    num_channels = 1  # 모노 오디오

    # RAW 파일 읽기
    with open(raw_filepath, 'rb') as raw_file:
        raw_data = raw_file.read()

    # WAV 파일 생성
    with wave.open(wav_filepath, 'wb') as wav_file:
        wav_file.setsampwidth(sample_width)
        wav_file.setframerate(sample_rate)
        wav_file.setnchannels(num_channels)

        # RAW 데이터를 WAV 파일에 쓰기
        wav_file.writeframes(raw_data)
        
def extract_features(audio_path):
    # 음성 신호 불러오기
    sr=16000
    audio, sr = librosa.load(audio_path, sr=sr)
    n_fft = 8192  # 원하는 FFT 윈도우 크기
    hop_length=4096
    n_mfcc=75
    fmin=0
    fmax=8000
    top_db=90

    # 스케일 조정
    max_amp = np.max(np.abs(audio))
    scale_factor = 1.0 / max_amp
    scaled_audio = audio * scale_factor
     
    # 스펙트로그램 평활화
    norm_audio = librosa.util.normalize(scaled_audio)
    
    # RMS 정규화
    rms = librosa.feature.rms(y=norm_audio)
    normalized_audio = norm_audio / (np.max(rms) + 1e-8)
  
    # MFCC 추출
    mfcc = librosa.feature.mfcc(y=normalized_audio, sr=sr, n_mfcc=n_mfcc, 
                                hop_length=hop_length, n_fft=n_fft, fmin=fmin, fmax=fmax)
    
    return mfcc


In [3]:
# CTL 파일 읽기
with open(ctl_file, 'r') as file:
    filelist = [line.strip().replace('\\', '/') + '.raw' for line in file]

#.raw형태의 테스트 데이터를 .wav로 바꿔서 저장할 폴더 생성
if os.path.exists(wav_folder) == False:
    os.mkdir(wav_folder)    

# RAW 파일을 WAV 파일로 변환
for fileinfo in filelist:
    raw_filepath = os.path.join(raw_folder, fileinfo)
    wav_filename = os.path.splitext(fileinfo)[0] + '.wav'
    wav_filepath = os.path.join(wav_folder, wav_filename)

    convert_raw_to_wav(raw_filepath, wav_filepath)

#wav파일이 저장된 경로와 확장자
folder_path = "202301ml_fmcc/raw16k/test/wavtest/"
extension = "*.wav"

#ctl파일에 저장된 .raw파일 확장자를 .wav로 바꾼 후 저장
with open(ctl_file, 'r') as file:
    filelist = [line.strip().replace('\\', '/') + '.wav' for line in file]

#테스트 데이터셋의 .wav 파일이 존재하는 경로명을 저장
tests_dataset=[]  
for file in filelist:
    raw_filepath = os.path.join(folder_path, file)
    wav_filename = os.path.splitext(file)[0] + '.wav'
    wav_filepath = os.path.join(wav_folder, wav_filename)
    tests_dataset.append(wav_filepath)    

#위의 .wav 파일의 경로를 통해 특성 추출
tests_features = []
for audio_path in tests_dataset:
    features = extract_features(audio_path)
    tests_features.append(features)
    
#위의 추출한 특성을 패딩
newtest_features=[]  
for i in tests_features:
    current_size = np.shape(i)[1]
    if current_size < max_size:
        padding_size = max_size - current_size
        padding = np.zeros((np.shape(i)[0], padding_size))
        padded_array = np.concatenate((i, padding), axis=1)
        newtest_features.append(padded_array)
    else:
        newtest_features.append(i)
    
#패딩한 특성을 다시 ndarray로 변환
tests_features = np.array(newtest_features)

In [4]:
# 테스트 데이터 전처리
flattened_test_features = tests_features.reshape(tests_features.shape[0], -1)
flattened_test_nanfeatures = np.nan_to_num(flattened_test_features, nan=0.0)

#LDA 정의
n_components = 1

# 학습용 코드에서 학습할 때 fit한 LDA를 이용하여 테스트 데이터에 대한 LDA 변환
lda_test_features = lda.transform(flattened_test_nanfeatures)

#학습코드에서 학습 후 저장한 모델을 불러옴
file_name = 'gmm_model.pkl'
gmm = joblib.load(file_name)

# 테스트 데이터에 대한 GMM 예측
prediction = gmm.predict(lda_test_features)

In [32]:
#
f = open("202301ml_fmcc/개구리애호가_test_results.txt", 'w', encoding='utf-8')
for i in range(len(tests_dataset)):
    if prediction[i] == 0:
        predict_gender = "male\r\n"
    elif prediction[i] == 1:
        predict_gender = "feml\r\n"
    result = str(tests_dataset[i]) + " " + predict_gender 
    f.write(result)

f.close()



[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 0 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 0 1 1 1 0 1 1 1 1 0 1 1 1 1
 1 1 0 1 1 0 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1
 1 1 1 1 1 0 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 